<a href="https://colab.research.google.com/github/shank10/Gemma/blob/main/GemmaTrainingToBetterCoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments,EvalPrediction,pipeline
from peft import LoraConfig, TaskType, get_peft_model , PeftModel, LoraModel
from accelerate.utils import release_memory
from accelerate import Accelerator
from datasets import Dataset
import evaluate
import accelerate

from sklearn.metrics import f1_score
from typing import Dict

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from trl import SFTTrainer

import torch
import torch.nn

import os
os.environ['TOKENIZERS_PARALLELISM'] = "FALSE"
import warnings
warnings.filterwarnings("ignore")
import wandb
import gc
gc.collect()
wandb.init()

In [ ]:
code = pd.read_csv("/kaggle/input/python-code-instruction-dataset/train.csv")
code = code.sample(frac = 0.1,random_state = 101,ignore_index=True)
qa = pd.read_csv("/kaggle/input/glaive-python-code-qa-dataset/train.csv",names = ['output','instruction'])
qa = qa.sample(frac = 0.01,random_state = 101,ignore_index=True)

code.info()

In [ ]:
kaggle datasets download -d thedevastator/python-code-instruction-dataset

In [ ]:
def preprocess_data(example):
    template = f"""Instruction:\n{example['instruction']}\n\nResponse:\n{example['output']}"""
    return template

code['template'] = code.apply(preprocess_data,axis=1)
qa['template'] = qa.apply(preprocess_data,axis=1)

dataset = Dataset.from_pandas(pd.concat([code[['template']],qa[['template']]])).train_test_split(test_size = 0.2)

# Example Printing
print(code['template'].iloc[1])

In [ ]:
model_structure = """This is the structure of Gemma Model,


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNorm()
  )
  (lm_head): Linear(in_features=2048, out_features=256000, bias=False)
)"""
print(model_structure)

In [ ]:
base_model = "/kaggle/input/gemma/transformers/2b-it/2"
lora_config = LoraConfig(
        lora_dropout=0.1,
        r=8,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='fp4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)

loraModel = LoraModel(model= model,config =lora_config,adapter_name = "adapter")

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(loraModel)

In [ ]:
torch.cuda.empty_cache()
trainer.train()


In [ ]:
lora_adapter_path = "LoraAdapter"
trainer.model.save_pretrained(lora_adapter_path)


In [ ]:
# Release Memory
trainer, model, = release_memory(trainer, model)
gc.collect()
torch.cuda.empty_cache()


In [ ]:
final_model = "Final_Model"
# Loading Base Model
model = AutoModelForCausalLM.from_pretrained(base_model,device_map='auto', torch_dtype=torch.float16)

# Lodaing  Base Model with LoRA adapters.
peft_model = PeftModel.from_pretrained(model,lora_adapter_path,device_map='auto', torch_dtype=torch.float16)

# Merging and Saving Model
model = peft_model.merge_and_unload(progressbar = True)
model.save_pretrained(final_model)
tokenizer.save_pretrained(final_model)


In [ ]:
model = release_memory(model)

In [ ]:
peft_pipe = pipeline("text-generation",final_model, model_kwargs={"torch_dtype": torch.float16},
    device_map='auto',
    max_new_tokens=512)


In [ ]:
def get_output(question):
    prompt = f"Provide an optimal Response to the given Instruction, dont include the any kind of extra Explaination in response. keep in short and simple.\n\nInstruction:\n{question}\n\nResponse:\n"
    out = peft_pipe(prompt,
     do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True)

    return out


In [ ]:
out = get_output("Write program to find factorial of number")
print(out[0]['generated_text'])


In [ ]:
out = get_output("Explain how can an AI classify a given sentence as a declaration, an expression, or a statement?")
print(out[0]['generated_text'])


In [ ]:
out = get_output("I need to write a bash script that compares two arguments and prints a message if the first argument is greater than the second argument. Can someone help me with the code")
print(out[0]['generated_text'])


In [ ]:
out = get_output("How can I optimize a TensorFlow neural network to improve its predictive accuracy?")
print(out[0]['generated_text'])
